In [ ]:
!pip install anvil-uplink

In [ ]:
import anvil.server
anvil.server.connect("server_HAQPFOVYCD4RPFILZ4SIBZY2-VOEAFQ7CLLD47XM6")

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.15.0


In [ ]:
import os
os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

In [ ]:
import os
from google.colab import files
import shutil

folder_name = 'CVAT'
os.makedirs(folder_name, exist_ok=True)

uploaded = files.upload()

for filename in uploaded.keys():
    shutil.move(filename, os.path.join(folder_name, filename))


In [ ]:
import matplotlib.pyplot as plt
import os
import numpy as np
from PIL import Image, ImageDraw
import xml.etree.ElementTree as ET  # For parsing XML
import zipfile

# Define  categories
categories = ["empty", "clear_fluid", "solid", "air"]

image_dir = "/content/CVAT"
image_filenames = os.listdir(image_dir)

# Define a list to store labels for each image
labels = []

for filename in image_filenames:
    if "empty" in filename or "air" in filename:
        label = [1, 0, 0, 0]  # Empty antrum
    elif "fluid" in filename:
        label = [0, 1, 0, 0]  # Clear fluid in the antrum
    elif "solid" in filename:
        label = [0, 0, 1, 0]  # Solids in the antrum

    labels.append(label)

labels = np.array(labels)

print(labels)

'''
# Print the labels and an image
for i, filename in enumerate(image_filenames):
    # Skip directories
    if os.path.isdir(os.path.join(image_dir, filename)):
        continue

    print("Image: ", filename)
    print("Label: ", labels[i])

    # Load and display the image
    image_path = os.path.join(image_dir, filename)
    img = Image.open(image_path)
    plt.imshow(img)
    plt.axis('off')  # Hide axes
    plt.show()
    '''

labeling and preprocessing images

In [ ]:
import matplotlib.pyplot as plt
import os
import numpy as np
from PIL import Image, ImageDraw
import xml.etree.ElementTree as ET  # For parsing XML
import zipfile

# Define  categories
categories = ["empty", "clear_fluid", "solid", "air"]

zip_file_path = "/content/segmentantrum11.29.zip"
extracted_folder_path = "/content/extracted_annotations"

# Extract zip file to a folder
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder_path)

# List the files in the extracted folder
extracted_filenames = os.listdir(extracted_folder_path)
image_dir = "/content/CVAT"

import os
import xml.etree.ElementTree as ET

xml_file_path = "/content/extracted_annotations/annotations.xml"
output_dir = "/content/extracted_annotations/indiv_xml"

os.makedirs(output_dir, exist_ok=True)

# Parse xml file
tree = ET.parse(xml_file_path)
root = tree.getroot()

# Iterate through the <image> xml elements
for image_elem in root.findall(".//image"):
    # Extract image attributes
    image_name = image_elem.get("name")

    # Create a new XML file for this image
    image_xml = ET.Element("annotations")
    version_elem = ET.SubElement(image_xml, "version")
    version_elem.text = "1.1"
    meta_elem = ET.SubElement(image_xml, "meta")
    image_elem_copy = ET.Element("image")
    image_elem_copy.set("name", image_name)
    meta_elem.append(image_elem_copy)

    # Find the annotations for this image
    annotations = root.findall(f".//image[@name='{image_name}']/*")

    # Add the annotations for this image to the new XML file
    for annotation in annotations:
        image_xml.append(annotation)

    # Save the new XML file with the same name as the image
    output_xml_path = os.path.join(output_dir, image_name.replace(" ", "_").replace(".", "_") + ".xml")
    tree = ET.ElementTree(image_xml)
    tree.write(output_xml_path)

print("Individual XML files have been created in:", output_dir)


simulation of ultrasound code, not run each time

In [ ]:
'''
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from keras.models import Sequential
from keras.layers import Dense, Reshape, Conv2DTranspose, Conv2D, Flatten, BatchNormalization, LeakyReLU

image_dir = "/content/CVAT"
def load_images_from_directory(image_dir):
    image_list = []
    for filename in os.listdir(image_dir):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            img_path = os.path.join(image_dir, filename)
            img = Image.open(img_path)
            img = img.convert('L')  # Grayscale
            img = img.resize((28, 28))  # Resize to match input shape
            img_array = np.array(img)
            img_array = img_array / 255.0  # Normalize pixel values
            img_array = (img_array - 0.5) / 0.5  # Standardize pixel values
            img_array = np.expand_dims(img_array, axis=-1)  # Add dimension
            image_list.append(img_array)
    return np.array(image_list)

def build_generator(latent_dim):
    model = Sequential()
    model.add(Dense(7 * 7 * 256, input_dim=latent_dim))
    model.add(Reshape((7, 7, 256)))
    model.add(Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.01))
    model.add(Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.01))
    model.add(Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', activation='tanh'))
    return model

def build_discriminator(input_shape):
    model = Sequential()
    model.add(Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=input_shape))
    model.add(LeakyReLU(alpha=0.01))
    model.add(Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.01))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    return model

def build_progan(latent_dim, input_shape):
    generator = build_generator(latent_dim)
    discriminator = build_discriminator(input_shape)
    discriminator.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    discriminator.trainable = False
    gan = Sequential()
    gan.add(generator)
    gan.add(discriminator)
    gan.compile(loss='binary_crossentropy', optimizer='adam')
    return gan

def generate_and_print_images(generator, num_images):
    latent_vectors = np.random.randn(num_images, latent_dim)
    generated_images = generator.predict(latent_vectors)

    plt.figure(figsize=(10, 10))
    for i in range(num_images):
        plt.subplot(3, 3, i + 1)
        plt.imshow(generated_images[i, :, :, 0], cmap='gray')
        plt.axis('off')
    plt.tight_layout()
    plt.show()

# Set hyperparameters
latent_dim = 100
input_shape = (28, 28, 1)

# Build the ProGAN model
progan = build_progan(latent_dim, input_shape)

# Generate and print 9 images
generate_and_print_images(progan.layers[0], 9)

'''

merging antrum annotations and images

In [ ]:
import cv2
import os
import xml.etree.ElementTree as ET
from IPython.display import Image, display

image_dir = "/content/CVAT
xml_dir = "/content/extracted_annotations/indiv_xml"
output_dir = "/content/outputtedCVAT"

# Create the output directory
os.makedirs(output_dir, exist_ok=True)
# Iterate through the XML annotation files
for xml_file in os.listdir(xml_dir):
    if xml_file.endswith(".xml"):
        xml_path = os.path.join(xml_dir, xml_file)

        # Determine the image format
        image_name_without_extension = os.path.splitext(xml_file)[0]
        jpg_image_path = os.path.join(image_dir, image_name_without_extension + ".jpg")
        png_image_path = os.path.join(image_dir, image_name_without_extension + ".png")

        if os.path.exists(jpg_image_path):
            image_path = jpg_image_path
        elif os.path.exists(png_image_path):
            image_path = png_image_path
        else:
            print(f"Image not found for XML: {xml_file}")
            continue  # Skip this XML file if the image is not found

        # Load the image
        img = cv2.imread(image_path)

        # Check if the image was loaded successfully
        if img is None:
            print(f"Failed to load image: {image_path}")
            continue  # Skip this image and move to the next one

        # Parse the XML file
        tree = ET.parse(xml_path)
        root = tree.getroot()

        for annotation_elem in root:
            if annotation_elem.tag == "ellipse":
                # Extract ellipse attributes
                cx = int(float(annotation_elem.get("cx")))
                cy = int(float(annotation_elem.get("cy")))
                rx = int(float(annotation_elem.get("rx")))
                ry = int(float(annotation_elem.get("ry")))
                label = annotation_elem.get("label")

                # Draw an ellipse on the image
                cv2.ellipse(img, (cx, cy), (rx, ry), 0, 0, 360, (0, 0, 255), 2)
                cv2.putText(img, label, (cx - 20, cy - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

            elif annotation_elem.tag == "polyline":
                # Extract polyline attributes
                points_str = annotation_elem.get("points")
                points = [int(float(coord.split(',')[0])) for coord in points_str.split(';')], \
                         [int(float(coord.split(',')[1])) for coord in points_str.split(';')]
                points = [(points[0][i], points[1][i]) for i in range(len(points[0]))]

                # Draw the polyline on the image
                cv2.polylines(img, [np.array(points, dtype=np.int32)], isClosed=False, color=(0, 0, 255), thickness=2)

                label = annotation_elem.get("label")
                if label:
                    label_x = points[0][0] - 20
                    label_y = points[1][0] - 20
                    cv2.putText(img, label, (label_x, label_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

        # Save the annotated image
        annotated_image_path = os.path.join(output_dir, image_name_without_extension + ".jpg")
        cv2.imwrite(annotated_image_path, img)

print("Annotated images have been saved in:", output_dir)
# Directory containing annotated images
output_dir = "/content/outputtedCVAT"

# List the files in the output directory
annotated_image_files = os.listdir(output_dir)
annotated_image_files = [file for file in os.listdir(output_dir) if os.path.isfile(os.path.join(output_dir, file))]

# Display the first few annotated images
num_images_to_display = min(len(annotated_image_files), 15)
for i in range(num_images_to_display):
    image_path = os.path.join(output_dir, annotated_image_files[i])
    display(Image(filename=image_path))


In [ ]:
!pip install albumentations

In [ ]:
!mkdir foldername

ViT Adaptor model

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageData
import matplotlib.pyplot as plt

dataset_dir = '/content/CVAT'

# Define image dimensions and batch size
img_width, img_height = 128, 128
batch_size = 8

# Data preprocessing and augmentation
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Load and augment the training dataset
train_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',  # Change class_mode to 'categorical'
    shuffle=True
)

# Define the CNN architecture
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(4, activation='softmax')
])

# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(learning_rate=1e-4),  # Use learning_rate instead of lr
              metrics=['accuracy'])

# Train the model
# h5 is one of the Hierarchical Data Formats (HDF) used to store large amount of data
history = model.fit(train_generator, epochs=24)
model.save('gastric_ultrasound_model.h5')

# Display some sample images from the dataset
def show_images(generator, num_images=5):
    plt.figure(figsize=(10, 10))
    for i in range(num_images):
        image = next(generator)[0]
        plt.subplot(1, num_images, i + 1)
        plt.imshow(image)
        plt.axis('off')
    plt.show()

# Display some sample images from the dataset
def show_images(generator, num_images=5):
    plt.figure(figsize=(10, 10))
    images, labels = next(generator)
    for i in range(min(num_images, len(images))):
        plt.subplot(1, num_images, i + 1)
        plt.imshow(images[i])
        plt.axis('off')
    plt.show()

# Display sample images after training
show_images(train_generator, num_images=5)


In [ ]:
import cv2
import os
import xml.etree.ElementTree as ET
import numpy as np
import tensorflow as tf
from IPython.display import Image, display
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras
from tensorflow.keras import layers
import albumentations as A
from tensorflow.keras.regularizers import l1, l2
from tensorflow.keras.layers import Dropout, BatchNormalization
from tensorflow.keras.optimizers import RMSprop

np.random.seed(45)
tf.random.set_seed(45)

image_dir = "/content/outputtedCVAT"
xml_dir = "/content/extracted_annotations/indiv_xml"
output_dir = "/content/mergedCVAT"

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

data_dir = output_dir

# Define batch size and number of epochs
batch_size = 32
epochs = 15

albumentations_transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, p=0.5),
])

# data generator using tf.data and Albumentations
@tf.function
def process_path(file_path):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (224, 224))  # Adjust the target size
    img = tf.cast(img, tf.float32) / 255.0

    img_np = img.numpy()

    img_np = albumentations_transform(image=img_np)['image']

    return img_np

train_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,  # Normalize pixel values to [0, 1]
    rotation_range=20,   # Randomly rotate images by up to 20 degrees
    width_shift_range=0.2,  # Randomly shift the width of images
    height_shift_range=0.2,  # Randomly shift the height of images
    horizontal_flip=True  # Randomly flip images horizontally
)

train_generator = train_datagen.flow_from_directory(
    data_dir,  # Path to the data directory
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
    seed=15
)

class SqueezeExciteBlock(layers.Layer):
    def __init__(self, ratio=8, **kwargs):
        super(SqueezeExciteBlock, self).__init__(**kwargs)
        self.ratio = ratio

    def build(self, input_shape):
        self.filters = input_shape[-1]
        self.se_reduce = layers.Conv2D(self.filters // self.ratio, kernel_size=1, activation='relu', strides=1, padding='same')
        self.se_expand = layers.Conv2D(self.filters, kernel_size=1, activation='sigmoid', strides=1, padding='same')

    def call(self, inputs):
        se_tensor = layers.GlobalAveragePooling2D()(inputs)
        se_tensor = tf.reshape(se_tensor, [-1, 1, 1, self.filters])
        se_tensor = self.se_reduce(se_tensor)
        se_tensor = self.se_expand(se_tensor)
        return inputs * se_tensor

class VisionTransformerAdapter(nn.Module):
    def __init__(self, input_channels, d_model, hidden_size, dropout_rate=0.1):
        super(VisionTransformerAdapter, self).__init__()

        self.conv1d = nn.Conv1d(input_channels + d_model, hidden_size, kernel_size=1)
        self.relu = nn.ReLU()
        self.feed_forward = nn.Linear(hidden_size, d_model)
        self.conv2d = nn.Conv2d(1, hidden_size, kernel_size=3, stride=1, padding=1)
        self.se_block = SqueezeExciteBlock(hidden_size)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x, positional_embedding):
        x = x + positional_embedding
        x = x.view(x.size(0), x.size(1), -1).permute(0, 2, 1)
        x = self.conv1d(x)
        x = self.relu(x)
        x = self.feed_forward(x)
        x = self.relu(x)
        x = x.unsqueeze(1)
        x = self.conv2d(x)
        x = self.se_block(x)  # Apply SE block
        x = self.relu(x)
        x = self.dropout(x)
        return x

    def call(self, inputs):
        se_tensor = tf.reduce_mean(inputs, axis=[1, 2], keepdims=True)
        se_tensor = self._se_reduce(se_tensor)
        se_tensor = tf.nn.relu(se_tensor)
        se_tensor = self._se_expand(se_tensor)
        se_tensor = tf.nn.sigmoid(se_tensor)
        return inputs * se_tensor

# Define the neural network architecture with Squeeze-and-Excitation
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    SqueezeExciteBlock(ratio=8),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(len(train_generator.class_indices), activation='softmax')
])

optimizer = RMSprop(learning_rate=0.00001)

model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',  # categorical crossentropy for one-hot encoded labels
              metrics=['accuracy'])

# Iterate through the XML annotation files
for xml_file in os.listdir(xml_dir):
    if xml_file.endswith(".xml"):
        xml_path = os.path.join(xml_dir, xml_file)

        # Determine the image format
        image_name_without_extension = os.path.splitext(xml_file)[0]
        jpg_image_path = os.path.join(image_dir, image_name_without_extension + ".jpg")
        png_image_path = os.path.join(image_dir, image_name_without_extension + ".png")

        if os.path.exists(jpg_image_path):
            image_path = jpg_image_path
        elif os.path.exists(png_image_path):
            image_path = png_image_path
        else:
            print(f"Image not found for XML: {xml_file}")
            continue  # Skip this XML file if the image is not found

        # Load the image
        img = cv2.imread(image_path)

        # Check if the image was loaded successfully
        if img is None:
            print(f"Failed to load image: {image_path}")
            continue  # Skip this image and move to the next one

        # Parse the XML file
        tree = ET.parse(xml_path)
        root = tree.getroot()

        for annotation_elem in root:
            if annotation_elem.tag == "ellipse":
                # Extract ellipse attributes
                cx = int(float(annotation_elem.get("cx")))
                cy = int(float(annotation_elem.get("cy")))
                rx = int(float(annotation_elem.get("rx")))
                ry = int(float(annotation_elem.get("ry")))
                label = annotation_elem.get("label")

                # Draw an ellipse on the image
                cv2.ellipse(img, (cx, cy), (rx, ry), 0, 0, 360, (0, 0, 255), 2)
                cv2.putText(img, label, (cx - 20, cy - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

            elif annotation_elem.tag == "polyline":
                # Extract polyline attributes
                points_str = annotation_elem.get("points")
                points = [int(float(coord.split(',')[0])) for coord in points_str.split(';')], \
                         [int(float(coord.split(',')[1])) for coord in points_str.split(';')]
                points = [(points[0][i], points[1][i]) for i in range(len(points[0]))]

                # Draw the polyline on the image
                cv2.polylines(img, [np.array(points, dtype=np.int32)], isClosed=False, color=(0, 0, 255), thickness=2)

                label = annotation_elem.get("label")
                if label:
                    label_x = points[0][0] - 20
                    label_y = points[1][0] - 20
                    cv2.putText(img, label, (label_x, label_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

        # Save the annotated image
        annotated_image_path = os.path.join(output_dir, image_name_without_extension + ".jpg")
        cv2.imwrite(annotated_image_path, img)

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=epochs,
    shuffle=False
)

test_datagen = ImageDataGenerator(rescale=1.0 / 255.0)
test_generator = test_datagen.flow_from_directory(
    data_dir,  # Path to the data directory
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

submitted_ultrasound_path = "/content/submittedUltrasound.png"
submitted_ultrasound = cv2.imread(submitted_ultrasound_path)
submitted_ultrasound = cv2.resize(submitted_ultrasound, (224, 224))
submitted_ultrasound = np.expand_dims(submitted_ultrasound, axis=0) / 255.0

# Make predictions on the submitted ultrasound image
predictions = model.predict(submitted_ultrasound)

# Convert predictions to class label
predicted_class = np.argmax(predictions)
print(f"Predicted Class for Submitted Ultrasound: {predicted_class}")

# Evaluate the model on the test set
test_datagen = ImageDataGenerator(rescale=1.0 / 255.0)
test_generator = test_datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

test_loss, test_acc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc}")

diameter

In [ ]:
import zipfile
import os

zip_file_path = '/content/diameterAnnotations.zip'
extracted_dir = '/content/extractions'

# Create the target directory if it doesn't exist
os.makedirs(extracted_dir, exist_ok=True)

# Open the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # Extract all the contents to the specified directory
    zip_ref.extractall(extracted_dir)

print(f"Contents extracted to: {extracted_dir}")

In [ ]:
import xml.etree.ElementTree as ET
import os
import cv2
import numpy as np
from google.colab.patches import cv2_imshow  # Import cv2_imshow for displaying images in Colab

# Function to parse the XML file and get annotations
def parse_annotations(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    annotations = []

    for image_elem in root.findall('image'):
        image_name = image_elem.get('name')
        image_annotations = []

        for polyline_elem in image_elem.findall('polyline'):
            label = polyline_elem.get('label')
            points_str = polyline_elem.get('points')
            points = [tuple(map(float, point.split(','))) for point in points_str.split(';')]

            image_annotations.append({
                'label': label,
                'points': points
            })

        annotations.append({
            'image_name': image_name,
            'annotations': image_annotations
        })

    return annotations

# print annotated images
def print_annotated_images(annotations, images_dir):
    for annotation in annotations:
        image_name = annotation['image_name']
        image_path = os.path.join(images_dir, image_name)

        # Load the image using OpenCV
        image = cv2.imread(image_path)

        if image is None:
            print(f"Error loading image: {image_path}")
            continue

        for polyline in annotation['annotations']:
            label = polyline['label']
            points = np.array(polyline['points'], dtype=np.int32)  # Convert points to int32

            # Draw polyline on the image
            cv2.polylines(image, [points], isClosed=True, color=(0, 255, 0), thickness=2)

        # Display the image with annotations
        cv2_imshow(image)

xml_file_path = '/content/extractions/annotations.xml'
images_dir = '/content/images'

annotations = parse_annotations(xml_file_path)
print_annotated_images(annotations, images_dir)

In [ ]:
import xml.etree.ElementTree as ET
import os
import cv2
import numpy as np
from tensorflow import keras
from tensorflow.keras.optimizers import Adam  # Import Adam optimizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# parse the XML file and get annotations
def parse_annotations(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    annotations = []

    for image_elem in root.findall('image'):
        image_name = image_elem.get('name')
        image_annotations = []

        for polyline_elem in image_elem.findall('polyline'):
            label = polyline_elem.get('label')
            points_str = polyline_elem.get('points')
            points = [tuple(map(float, point.split(','))) for point in points_str.split(';')]

            image_annotations.append({
                'label': label,
                'points': points
            })

        annotations.append({
            'image_name': image_name,
            'annotations': image_annotations
        })

    return annotations

# Load annotations
xml_file_path = '/content/extractions/annotations.xml'
images_dir = '/content/CVAT'

annotations = parse_annotations(xml_file_path)

X = []  # Images
labels = []  # List of Labels

for annotation in annotations:
    image_name = annotation['image_name']
    image_path = os.path.join(images_dir, image_name)

    # Load the image using OpenCV
    image = cv2.imread(image_path)

    if image is None:
        print(f"Error loading image: {image_path}")
        continue

    for polyline in annotation['annotations']:
        label_value = polyline['label']  # Use a different variable name
        points = np.array(polyline['points'], dtype=np.int32)

        # Extract the region of interest (ROI) from the image based on the points
        x, y, w, h = cv2.boundingRect(points)
        roi = image[y:y+h, x:x+w]

        # Resize the ROI to a fixed size
        roi = cv2.resize(roi, (224, 224))

        X.append(roi)
        labels.append(label_value)  # Append the label value

X = np.array(X)
labels = np.array(labels)

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(labels)

X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Define the CNN model with Adam optimizer and a learning rate of 0.001
model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(len(label_encoder.classes_), activation='softmax')  # Output layer
])

# Set the learning rate for the Adam optimizer
adam_optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=adam_optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=28, validation_data=(X_test, y_test))

# Save the trained model
model.save('diameter_detection_model.h5')

# Load the submitted ultrasound image
submitted_ultrasound_path = '/content/submittedUltrasound.png'
submitted_ultrasound = cv2.imread(submitted_ultrasound_path)

# Preprocess the submitted ultrasound image
submitted_ultrasound = cv2.resize(submitted_ultrasound, (224, 224))

# Expand dimensions to match the input shape expected by the model
submitted_ultrasound = np.expand_dims(submitted_ultrasound, axis=0)

# Make predictions using the loaded model
predictions = model.predict(submitted_ultrasound)

# Decode the predicted labels using the label encoder
predicted_labels = label_encoder.inverse_transform(np.argmax(predictions, axis=1))

# Display or save the annotated image
cv2.imshow('Annotated Ultrasound', submitted_ultrasound[0])
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
import math

access_diameters = input("Use your ultrasound's scale to determine the length of the AP and CC diameters above. Type proceed when complete.  ").lower()

def calculate_gastric_volume():

    age_input = input("Enter patient age: ")
    ap_diameter_input = input("Measure the anteroposterior diameter of the antrum and enter value: ")
    cc_diameter_input = input("Measure the craniocaudal diameter of the antrum and enter value: ")

    try:
        age = float(age_input)
        ap_diameter = float(ap_diameter_input)
        cc_diameter = float(cc_diameter_input)

        # Calculate gastric volume
        gastric_volume = 27 + (15.6 * (math.pi * ap_diameter * cc_diameter / 4)) - (1.28 * age)

        return gastric_volume
    except ValueError:
        print("Invalid input. Please enter numeric values.")
        return None

if access_diameters:
  gastric_volume = calculate_gastric_volume()


In [ ]:
if predicted_class == 1 or predicted_class == 4:
  print("Empty antrum. Proceed with operation.")
elif predicted_class == 2:
  print("Clear fluid present in antrum.")
  if gastric_volume <= 1.5:
    print("Gastric volume is <= 1.5ml/kg. Proceed with operation.")
  else:
    print("Inoperable conditions. High risk of aspiration. If surgery is urgent or emergent, proceed with full stomach precaution.")
elif predicted_class == 3:
  print("Inoperable conditions, undigested solids are present in stomach. High risk of aspiration. If surgery is urgent or emergent, proceed with full stomach precaution.")
